
<style>
/* ------------------------------------------------------------------
   proxywhirl — Header "Rotating Proxy • Luminous Brand Text"
   STYLE-ONLY (plus lighter wording): Title trimmed, subtitle fresh.
   - Unified brand text for <code>…</code> (title & subtitle)
   - High-contrast gradient type, glossy highlight, data-stream underline
   - Rotating relay rings & faint network grid in the background
   - Dark-mode tuned, motion-respectful
-------------------------------------------------------------------*/

/* Fonts */
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;800&family=JetBrains+Mono:wght@600;700&display=swap');

/* ===========================================================
   Theme Tokens
=========================================================== */
:root{
  /* Accents (no #6a9fb5) */
  --pw-accent:   #8b5cf6; /* violet-500 */
  --pw-accent-2: #22d3ee; /* cyan-400  */
  --pw-accent-3: #f59e0b; /* amber-500 */
  --pw-accent-4: #ef4444; /* red-500   */

  /* Foreground */
  --pw-text:   #0f172a;   /* slate-900 */
  --pw-subtle: #334155;   /* slate-600 */

  /* Surfaces */
  --pw-card:   rgba(255, 255, 255, 0.78);
  --pw-border: rgba(148, 163, 184, 0.26);
  --pw-shadow: 0 18px 56px rgba(2, 6, 23, 0.18), 0 6px 16px rgba(2, 6, 23, 0.10);

  /* FX */
  --pw-radius: 22px;
  --pw-blur:   12px;
  --pw-sat:    1.20;

  /* Brand gradient */
  --brand-grad: linear-gradient(90deg,
    var(--pw-accent),
    var(--pw-accent-2),
    var(--pw-accent-3),
    var(--pw-accent-4),
    var(--pw-accent));

  /* Ring sizing */
  --ring-size: min(56vmin, 520px);
  --ring-thick: 10px;
}

@media (prefers-color-scheme: dark){
  :root{
    --pw-text:   #e2e8f0;  /* slate-200 */
    --pw-subtle: #94a3b8;  /* slate-400 */
    --pw-card:   rgba(2, 6, 23, 0.64);
    --pw-border: rgba(148, 163, 184, 0.20);
    --pw-shadow: 0 22px 60px rgba(0,0,0,.55), 0 8px 20px rgba(0,0,0,.35);
  }
}

/* ===========================================================
   Motion & Micro-Effects
=========================================================== */
@keyframes float{0%,100%{transform:translateY(0)}50%{transform:translateY(-6px)}}
@keyframes fadeInUp{from{opacity:0;transform:translateY(10px)}to{opacity:1;transform:translateY(0)}}
@keyframes shimmer{0%{background-position:0% 50%}100%{background-position:220% 50%}}
@keyframes spinCW{to{transform:rotate(360deg)}}
@keyframes spinCCW{to{transform:rotate(-360deg)}}
@keyframes scanlines{0%{background-position:0 0,0 0}100%{background-position:0 40px,0 0}}
@keyframes dataFlow{0%{background-position:0 0,0 0}100%{background-position:200% 0,0 0}}
@keyframes chromaPulse{
  0%,100%{filter:drop-shadow(0 8px 20px rgba(139,92,246,.20))}
  50%{filter:drop-shadow(0 10px 24px rgba(34,211,238,.26))}
}

/* ===========================================================
   Container
=========================================================== */
div[align='center']{
  font-family:'Inter',ui-sans-serif,system-ui,-apple-system,Segoe UI,Roboto,Helvetica,Arial,"Apple Color Emoji","Segoe UI Emoji";
  max-width:980px;
  margin:46px auto 26px;
  padding:42px 34px 26px;
  color:var(--pw-text);

  border:1px solid transparent;
  border-radius:var(--pw-radius);
  background:
    linear-gradient(180deg,var(--pw-card),var(--pw-card)) padding-box,
    linear-gradient(118deg,
      rgba(139,92,246,0.60),
      rgba(34,211,238,0.48),
      rgba(245,158,11,0.48),
      rgba(239,68,68,0.42)) border-box;
  box-shadow:var(--pw-shadow);

  position:relative;
  overflow:hidden;
  isolation:isolate;
  backdrop-filter:blur(var(--pw-blur)) saturate(calc(var(--pw-sat) * 115%));
  -webkit-backdrop-filter:blur(var(--pw-blur)) saturate(calc(var(--pw-sat) * 115%));
  animation:fadeInUp .55s ease-out both;
  text-rendering:geometricPrecision;
  -webkit-font-smoothing:antialiased;
}

/* Background layers: aurora + faint grid */
div[align='center']::before,
div[align='center']::after{
  content:"";
  position:absolute;
  inset:-1px;
  pointer-events:none;
  z-index:-1;
}
div[align='center']::before{
  background:
    radial-gradient(1100px 520px at 50% -18%, rgba(139,92,246,0.22), transparent 58%),
    radial-gradient(1000px 480px at 50% 118%, rgba(34,211,238,0.16), transparent 55%),
    radial-gradient(1200px 460px at -8% 40%, rgba(239,68,68,0.12), transparent 60%),
    radial-gradient(1100px 440px at 108% 60%, rgba(245,158,11,0.12), transparent 60%),
    repeating-linear-gradient(0deg, rgba(148,163,184,0.06) 0 1px, transparent 1px 28px),
    repeating-linear-gradient(90deg, rgba(148,163,184,0.05) 0 1px, transparent 1px 28px);
  animation: scanlines 16s linear infinite;
  background-size:auto,auto,auto,auto, auto 40px, 40px auto;
}

/* Conic wash */
div[align='center']::after{
  background:conic-gradient(from 160deg at 50% 50%,
    rgba(139,92,246,0.28),
    rgba(34,211,238,0.28),
    rgba(245,158,11,0.28),
    rgba(139,92,246,0.28));
  -webkit-mask:radial-gradient(closest-side, rgba(255,255,255,0.32), transparent 72%);
  mask:radial-gradient(closest-side, rgba(255,255,255,0.32), transparent 72%);
  filter:blur(22px);
  opacity:.55;
}

/* ===========================================================
   Rotating Proxy Rings (behind the title)
=========================================================== */
@supports (mask:radial-gradient(black, transparent)){
  div[align='center'] h1{position:relative; text-wrap:balance}
  div[align='center'] h1::before,
  div[align='center'] h1::after{
    content:"";
    position:absolute;
    left:50%;
    top:calc(-1 * (var(--ring-size) * 0.62));
    width:var(--ring-size);
    height:var(--ring-size);
    transform:translateX(-50%);
    pointer-events:none;
    z-index:-1;
  }
  /* Outer dashed ring */
  div[align='center'] h1::before{
    background:
      conic-gradient(from 0deg,
        rgba(255,255,255,0.0) 0 10deg,
        rgba(255,255,255,0.85) 10deg 12deg,
        rgba(255,255,255,0.0) 12deg 22deg) 50%/100% 100% no-repeat,
      var(--brand-grad);
    -webkit-mask:
      radial-gradient(circle, transparent calc(50% - var(--ring-thick)), #000 calc(50% - var(--ring-thick) + 1px)) intersect,
      radial-gradient(circle, #000 52%, transparent 52.3%);
    mask:
      radial-gradient(circle, transparent calc(50% - var(--ring-thick)), #000 calc(50% - var(--ring-thick) + 1px)) intersect,
      radial-gradient(circle, #000 52%, transparent 52.3%);
    border-radius:50%;
    box-shadow:0 0 28px rgba(139,92,246,0.25), inset 0 0 24px rgba(34,211,238,0.18);
    animation:spinCW 26s linear infinite;
  }
  /* Inner dotted ring */
  div[align='center'] h1::after{
    width:calc(var(--ring-size) * 0.74);
    height:calc(var(--ring-size) * 0.74);
    top:calc(-1 * (var(--ring-size) * 0.50));
    background:
      radial-gradient(circle at 50% 0%, rgba(255,255,255,0.95) 0 3px, transparent 3.5px) 50% 50%/6px 6px repeat,
      var(--brand-grad);
    -webkit-mask:
      radial-gradient(circle, transparent calc(50% - (var(--ring-thick) * .75)), #000 calc(50% - (var(--ring-thick) * .75) + 1px)) intersect,
      radial-gradient(circle, #000 52%, transparent 52.2%);
    mask:
      radial-gradient(circle, transparent calc(50% - (var(--ring-thick) * .75)), #000 calc(50% - (var(--ring-thick) * .75) + 1px)) intersect,
      radial-gradient(circle, #000 52%, transparent 52.2%);
    border-radius:50%;
    filter:drop-shadow(0 0 14px rgba(34,211,238,0.28));
    animation:spinCCW 18s linear infinite;
  }
}

/* ===========================================================
   Logo
=========================================================== */
div[align='center'] img{
  display:block;
  margin:12px auto 14px;
  transform-origin:center;
  animation:float 6s ease-in-out infinite;
  filter:drop-shadow(0 12px 30px rgba(139,92,246,0.34));
}

/* ===========================================================
   Title & Subtitle
=========================================================== */
div[align='center'] h1{
  margin:12px 0 10px;
  font-weight:800;
  font-size:clamp(32px, 3.6vw, 48px);
  letter-spacing:-0.02em;
  line-height:1.16;
  color:var(--pw-text);
}
div[align='center'] p{
  margin:6px 0 18px;
  color:var(--pw-subtle);
  font-size:clamp(13px, 1.6vw, 16px);
  letter-spacing:0.01em;
  animation:fadeInUp .65s ease-out both .05s;
}

/* ===========================================================
   UNIFIED BRAND TEXT — <code>…</code> (title & subtitle)
   SAME DESIGN • title = larger/stronger
=========================================================== */
div[align='center'] h1 code,
div[align='center'] p code{
  position:relative;
  display:inline-block;
  vertical-align:middle;

  font-family:'JetBrains Mono',ui-monospace,SFMono-Regular,Menlo,Consolas,"Liberation Mono",monospace;
  letter-spacing:.0065em;

  background:var(--brand-grad);
  background-size:260% 100%;
  background-clip:text;
  -webkit-background-clip:text;
  color:transparent;
  -webkit-text-fill-color:transparent;
  animation:shimmer 12s linear infinite;

  -webkit-text-stroke:.012em rgba(2,6,23,.40);
  text-shadow:
    0 .8px 0 rgba(255,255,255,.18),
    0 2px 10px rgba(139,92,246,.22),
    0 8px 22px rgba(34,211,238,.12),
    0.6px 0 rgba(239,68,68,.10),
   -0.6px 0 rgba(34,211,238,.10);
  filter:saturate(1.12);
  transform:translateZ(0);
}
div[align='center'] h1 code{
  font-weight:800;
  -webkit-text-stroke-width:.020em;
  animation:shimmer 11s linear infinite, chromaPulse 4.6s ease-in-out infinite;
}
div[align='center'] p code{
  font-weight:700;
  -webkit-text-stroke-width:.014em;
}

/* Gloss highlight (both) */
div[align='center'] h1 code::before,
div[align='center'] p code::before{
  content:"";
  position:absolute; inset:-6% -8%;
  border-radius:10px;
  background:
    linear-gradient(120deg,
      rgba(255,255,255,0.00) 35%,
      rgba(255,255,255,0.20) 48%,
      rgba(255,255,255,0.00) 62%);
  mix-blend-mode:screen;
  opacity:.65;
  filter:blur(2px);
  pointer-events:none;
}

/* Data-stream underline (both) */
div[align='center'] h1 code::after,
div[align='center'] p code::after{
  content:"";
  position:absolute;
  left:-2%;
  right:-2%;
  bottom:-.18em;
  height:2px;
  border-radius:999px;
  background:
    linear-gradient(90deg,
      rgba(255,255,255,.0) 0%,
      rgba(255,255,255,.85) 8%,
      rgba(255,255,255,.0) 16%) 0 0/140px 100% repeat-x,
    var(--brand-grad);
  opacity:.9;
  animation:dataFlow 2.4s linear infinite, shimmer 12s linear infinite;
  mix-blend-mode:screen;
  pointer-events:none;
}

/* Hover micro-lift */
@media (hover:hover) and (pointer:fine){
  div[align='center'] h1 code:hover,
  div[align='center'] p code:hover{
    transform:translateY(-1px);
    text-shadow:
      0 .8px 0 rgba(255,255,255,.22),
      0 4px 18px rgba(139,92,246,.38),
      0 16px 32px rgba(34,211,238,.18),
      0.6px 0 rgba(239,68,68,.18),
     -0.6px 0 rgba(34,211,238,.18);
  }
}

/* Selection colors */
div[align='center'] code::selection{
  background:rgba(139,92,246,.25);
  -webkit-text-fill-color:#fff;
}
div[align='center'] ::selection{
  background:rgba(34,211,238,.18);
}

/* ===========================================================
   Divider — prism ribbon
=========================================================== */
div[align='center'] hr{
  height:2px;border:0;margin-top:22px;
  background:linear-gradient(90deg,transparent,
    rgba(139,92,246,.82),
    rgba(34,211,238,.82),
    rgba(245,158,11,.82),
    transparent);
  border-radius:999px;
  position:relative;
  animation:fadeInUp .75s ease-out both .1s;
}
div[align='center'] hr::after{
  content:"";
  position:absolute;inset:-8px 0 0 0;
  background:radial-gradient(600px 30px at 50% 0%, rgba(139,92,246,.36), transparent 60%);
  filter:blur(10px);pointer-events:none;
}

/* ===========================================================
   A11y — Respect motion preferences
=========================================================== */
@media (prefers-reduced-motion: reduce){
  *{animation:none !important; transition:none !important}
}
</style>

<div align='center'>
  <img src="./docs/public/img/icon.svg" width="15%"/>
  <h1><code>proxywhirl</code> Examples</h1>
  <p>Practical usage demos for <code>proxywhirl</code></p>
  <hr/>
</div>

---

## Setup

In [4]:
# uncomment if in active developent for hot reload
%load_ext autoreload
%aimport proxywhirl
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

## Python API

In [ ]:
from proxywhirl import ProxyWhirl

pw = ProxyWhirl()
pw.get_proxy()

2025-08-21 21:54:00.617 | INFO     | proxywhirl.validator:__init__:282 - Enhanced ProxyValidator initialized with %d test URLs
2025-08-21 21:54:00.617 | INFO     | proxywhirl.proxywhirl:_create_loaders:262 - Created 6 proxy loaders
2025-08-21 21:54:00.618 | INFO     | proxywhirl.proxywhirl:__init__:213 - ProxyWhirl initialized with memory cache and round_robin rotation
2025-08-21 21:54:00.618 | DEBUG    | proxywhirl.proxywhirl:_run_async:392 - ProxyWhirl sync method called in async context (e.g., Jupyter). Attempting nest_asyncio compatibility.
2025-08-21 21:54:00.619 | INFO     | proxywhirl.proxywhirl:get_proxy_async:557 - Cache is empty, auto-fetching proxies...
2025-08-21 21:54:00.620 | INFO     | proxywhirl.proxywhirl:fetch_proxies_async:464 - Loading proxies from the-speedx-http
2025-08-21 21:54:00.617 | INFO     | proxywhirl.proxywhirl:_create_loaders:262 - Created 6 proxy loaders
2025-08-21 21:54:00.618 | INFO     | proxywhirl.proxywhirl:__init__:213 - ProxyWhirl initialized wit

NoneType